### Julia Syntax
If you're familiar with Python and MATLAB, Julia syntax is a combination with some extra features and syntax sugar.

### Variables and Types
Julia is strongly typed, but providing type information is optional. The compiler will try to infer type information, and will throw an error when it fails.

In [2]:
x = 3;
y::Int64 = 4;

In [3]:
# If the compiler can convert the type it will do so automatically
a::Float64 = 3

3

In [4]:
# This will throw an error because 3.4 cannot be converted to Int64
z::Int64 = 3.4

InexactError: InexactError: Int64(3.4)

Julia lets you use any unicode character to define variables. Just type `\alpha` and hit `tab`. For underscores type `c\_p` and hit tab. Note that not all subscripts and superscripts are supported.

In [25]:
κ = 2
L = 3
cₚ = 4
ρ = 5
t = 1

α = κ / (cₚ * ρ)
Fo = α * t / (L^2)

0.011111111111111112

### Functions
Julia functions are defined with the `function` keyword and wrapped with `end`. Function specializations are created at compile time depending on the data passed to the function

In [5]:
function mult(x, y)
    return x * y
end

mult (generic function with 1 method)

In [6]:
z_int = mult(1,2)
z_float = mult(1.0, 2.0)
println("Integer multiplication: $(z_int) with type $(typeof(z_int))")
println("Float multiplication: $(z_float) with type $(typeof(z_float))")


Integer multiplication: 2 with type Int64
Float multiplication: 2.0 with type Float64


Type information can be set by the user as well. In practice, multiple `add` functions can be defined with different type information associated to their parameters. This is called multiple dispatch and will be covered in depth in the next section.

In [7]:
function add(x::Int, y::Int)
    println("Adding integers")
    return x + y
end

function add(x::Float64, y::Float64)
    println("Adding floats")
    return x + y
end

add (generic function with 2 methods)

In [8]:
z_int = add(1, 2);
z_float = add(1.0, 2.0);

Adding integers
Adding floats


Julia also supports multiple return values. You can enforce the type of the return variables but it is recomended to allow the compiler to determine their types.

In [9]:
function multiple_return(x, y)
    x_plus_y = x + y
    x_times_y = x * y
    return x_plus_y, x_times_y
end

multiple_return (generic function with 1 method)

In [10]:
z1, z2 = multiple_return(3, 4)
println("Sum: $(z1), Product: $(z2)")

Sum: 7, Product: 12


In [11]:
# Small functions can be defined in one line
square(x) = x * x
square(2)

4

In Julia, if data passed into a function is not copied to save memory. Therefore, modifications within the function will modify the data outside of the function as well. If your function modifies parameters it is customary to end the function with `!`.

In [44]:
function modifies_x!(x)
    x[1] = 336
    y = [1,2,3]
    return x + y
end

x = [1,2,3]

println("Before: $(x)")
modifies_x!(x)
println("After: $(x)")

Before: [1, 2, 3]
After: [336, 2, 3]


### Loops & Conditionals
The syntax here is very similar to Python without the `:`, you just need an `end` to denote the end of the block.

In [12]:
# For loops
for i in range(1, 3)
    print("$i ")
end
println()

for element in [1, 2, 3]
    print("$(element) ")
end
println()

# While loops
counter = 1
while counter <= 3
    print("$(counter) ")
    counter += 1
end

1 2 3 
1 2 3 
1 2 3 

Similarly, the if-else syntax is similar to Python. Just remove the `:` and add an `end`

In [13]:
function test(x,y)
    if x < y
        relation = "less than"
    elseif x == y
        relation = "equal to"
    else
        relation = "greater than"
    end
    println("x is ", relation, " y.")
end

test(1, 2)
test(1,1)

x is less than y.
x is equal to y.


### Custom Types : Structs
Structs are like classes in Python and MATLAB. By default structs are immutable. Providing type information in a struct common.

In [14]:
struct MyType
    a::Int
    b::Float64
end

In [15]:
MyType(3, 1.0)

MyType(3, 1.0)

In [16]:
MyType(3.4, 1.0)

InexactError: InexactError: Int64(3.4)

In Python you need to definie `__init__` which tells Python how to construct your object. Julia will always create a default constructor if one is not provided. To create an extra constructor, define a function with the same name as your `struct`.

In [17]:
# Constructor that handles the case when only one parameter is known
function MyType(a)
    return MyType(a,a)
end

MyType

In [18]:
MyType(3)

MyType(3, 3.0)

To make your structs more flexible you can use parametric types.

In [20]:
struct ParametricType{T}
    x1::T
    x2::T
end

In [21]:
p_int = ParametricType(1, 2)
println(typeof(p_int))
p_float = ParametricType(1.0, 2.0)
println(typeof(p_float))

ParametricType{Int64}
ParametricType{Float64}


### Broadcasting
Like in MATLAB you can broadcast operations like, `+`, with the `.` syntax. In Julia you can also broadcast functions with the `.` syntax.

In [30]:
x_vals = ones(4)
x_vals = x_vals .+ 2

4-element Vector{Float64}:
 3.0
 3.0
 3.0
 3.0

In [31]:
x_vals .+= [3,4,5,6]

4-element Vector{Float64}:
 6.0
 7.0
 8.0
 9.0

In [33]:
y_vals = ones(4)
y_vals .+= 2

4-element Vector{Float64}:
 3.0
 3.0
 3.0
 3.0

In [35]:
function add_one(x)
    return x + 1
end

z_vals = [1,2,3,4]
# The function add_one is broadcast over the array, z_vals.
add_one.(z_vals)

4-element Vector{Int64}:
 2
 3
 4
 5

### File I/O

In [36]:
# Writing files
outpath = joinpath(@__DIR__, "data", "write_test.txt")
open(outpath, "w") do file
    write(file, "Hello, World!")
end

13

There is a package made by Julia called `DelimitedFiles` which operates similar to `np.loadtxt`. Here we use the default functionality of Julia to parse a file. This example brings together a lot of what we have learned so far.

In [40]:
# Reading files
function parse_file(inpath::String)
    data = []
    open(inpath, "r") do file
        # eachline returns an iterator over lines in the file
        # this avoids loading the entire file into memory. 
        for line in eachline(file)
            # stip() removes whitespace
            line = strip(line)

            # Checks if the line starts
            if startswith(line, "#")
                println("Ignoring Comment: ", line)
                continue
            else
                # split() converts the line into an array, splitting on whitespace
                vals = split(line)
                # parse() is broadcast over the elements of vals to convert them to Float32
                # push!() adds the parsed values to the vector, data.
                push!(data, parse.(Float32, vals))
            end

        end
    end
    return data
end

inpath = joinpath(@__DIR__, "data", "read_test.txt")
parse_file(inpath)

Ignoring Comment: # Comment Line


3-element Vector{Any}:
 Float32[1.0, 1.0, 2.0, 3.0]
 Float32[2.0, 3.2, 3.2, 1.2]
 Float32[3.0, 12.2, 30.1, 19.2]